In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
import pandas as pd
import random
import torch, torchtext
from torchtext import data
import googletrans
import spacy
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from googletrans import Translator

In [ ]:
#@title This section loads diffrent objects needed
sp = spacy.load('en_core_web_sm')
STOPWORDS = sp.Defaults.stop_words
translator = Translator()
translator.raise_Exception = True
lang_list = list(googletrans.LANGUAGES.keys()) 

In [ ]:
import string 
punctuations = list(string.punctuation)
punctuations.append("''")

## Penn Treebank Parser

In [ ]:
import pytreebank
out_path = 'sst_{}.txt'
dataset = pytreebank.load_sst('/content/sample_data/sst_raw')

# Store train, dev and test in separate files
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))
# Print the length of the training set
print(len(dataset['train']))

## Data Augumentation

In [ ]:
def back_translate(sentence):
  translator = Translator()
  words = word_tokenize(sentence)
  if len(words) >= 2:
    trans_lang = random.choice(lang_list) 
    trans_text = translator.translate(sentence, dest=trans_lang).text
    reverse_trans = translator.translate(trans_text, src=trans_lang, dest='en').text 
    return reverse_trans
  else:
    return sentence  

def random_deletion(sentence, p=0.1): 
    words = word_tokenize(sentence)
    if len(words) == 1: # return if single word
        return sentence
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return sentence
    else:
        return (" ").join(remaining)

def random_swap(sentence, n=1): 
    words = word_tokenize(sentence)
    #print(words)
    if len(words) <= 2:
      return sentence
    else:
      length = range(len(words)) 
      for _ in range(n):
          idx1, idx2 = random.sample(length, 2)
          words[idx1], words[idx2] = words[idx2], words[idx1] 
      return (" ").join(words)

In [ ]:
def Augment_train_data(train_data):
  No_train = train_data.Label.count()
  print("Number of training sampels before augmnet:",No_train)
  count = 0
  b = 0
  r_s = 0
  r_d = 0  
  new_train_data = []
  for i in range(0,No_train):
    #print(train_data.Label[i])
    if train_data.Label[i] == 1:
      count+=1
      back_trans = back_translate(train_data.Senetence[i])
      if not back_trans == train_data.Senetence[i]:
        new_train_data.append([new_train.Label[i],back_trans])
        b+=1

      rand_swap = random_swap(train_data.Senetence[i])
      if not rand_swap == train_data.Senetence[i]:
        new_train_data.append([train_data.Label[i],rand_swap])
        r_s+=1
    
      rand_del = random_deletion(train_data.Senetence[i])
      if not rand_del == train_data.Senetence[i]:
        new_train_data.append([train_data.Label[i],rand_del])
        r_d+=1 
    if(count > 450):
      break  

  new_df = train_data.append(pd.DataFrame(new_train_data,columns=['Label','Senetence']),ignore_index = True)
  print("Number of training sampels after augmnet:",new_df.Senetence.count())
  print("Number of training sampels back translated:",b)
  print("Number of training sampels random swapped:",r_s)
  print("Number of training sampels random deleted:",r_d)

  return new_df,new_train_data

## Loading Dataset

In [ ]:
train_aug = pd.read_csv("Output_files\Augmented_train_full.csv",sep='|')

In [ ]:
valid = pd.read_csv("Output_files\dev.csv",sep='|')

## Data Preprocessing

In [ ]:
def preprocess(sentence):
    text_tokens = word_tokenize(sentence)
    tokens_filtered = [i.strip("".join(punctuations)) for i in text_tokens if i not in punctuations]
    tokens_filtered = (" ").join(tokens_filtered)
    #text_tokens = word_tokenize(tokens_filtered)
    #tokens_filtered= [word for word in text_tokens if not word in STOPWORDS]
    return tokens_filtered

In [ ]:
for i in train_aug.index:
    x = preprocess(train_aug.Senetence[i])
    if len(x) == 0:
        train_aug.at[i, "Senetence"] = train_aug.Senetence[i]
    else:
        train_aug.at[i, "Senetence"] = x

In [ ]:
for i in valid.index:
    x = preprocess(valid.Senetence[i])
    if len(x) == 0:
        valid.at[i, "Senetence"] = valid.Senetence[i]
    else:
        valid.at[i, "Senetence"] = x

## From Here the network portion starts

In [ ]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
Review = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [ ]:
fields = [('review', Review), ('label', Label)]

In [ ]:
example = [torchtext.legacy.data.Example.fromlist([train_aug.Senetence[i],train_aug.Label[i]], fields) for i in range(train_aug.shape[0])] 

In [ ]:
example_2 = [torchtext.legacy.data.Example.fromlist([valid.Senetence[i],valid.Label[i]], fields) for i in range(valid.shape[0])] 

## To Write data to pickels for future use

In [ ]:
import os, pickle
with open('/content/sample_data/data/train_data.pkl', 'wb') as tokens: 
    pickle.dump(example, tokens)
with open('/content/sample_data/data/valid_data.pkl', 'wb') as tokens: 
    pickle.dump(example_2, tokens)

In [ ]:
import os, pickle
with open('/content/New/output/train_data.pkl', 'rb') as tokens: 
    example = pickle.load(tokens)
with open('/content/New/output/valid_data.pkl', 'rb') as tokens: 
    example_2 = pickle.load(tokens)

In [ ]:
Stanford_dataset_train = torchtext.legacy.data.Dataset(example, fields)
Stanford_dataset_valid = torchtext.legacy.data.Dataset(example_2, fields)

In [ ]:
len(Stanford_dataset_train), len(Stanford_dataset_valid)

In [ ]:
vars(Stanford_dataset_train.examples[278])

In [ ]:
Review.build_vocab(Stanford_dataset_train)
Label.build_vocab(Stanford_dataset_train)

In [ ]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

In [ ]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((Stanford_dataset_train, Stanford_dataset_valid), batch_size = 128, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [ ]:
next(iter(train_iterator))
#len(train.examples[11].tweet)

In [ ]:
import os, pickle
with open('pickles_files\\tokenizer_aug.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,                           
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        #self.fc1 = nn.Linear(hidden_dim, hidden_fcn) #one more fully connected layer

        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        hidden = self.dropout(hidden)  
        
        #one more Fully connected layer
        dense_outputs = self.fc2(hidden)

        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [ ]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 100
num_hidden_nodes = 50
#num_hidden_nodes_fcn = 50
num_output_nodes = 5
num_layers = 2
dropout = 0.3

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [ ]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
import torch.optim as optim
#CUDA_LAUNCH_BLOCKING=1
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.review  
        
        # convert to 1D tensor
        predictions = model(review, review_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.review
            
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
N_EPOCHS = 100
best_valid_loss = float('inf')
train_loss_list = []
valid_loss_list = []
train_acc_list = []
valid_acc_list = []
for epoch in range(N_EPOCHS):
     
    # train the model
    print("Epoch:",epoch,flush=True)
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    # evaluate the model    
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    valid_loss_list.append(valid_loss)
    valid_acc_list.append(valid_acc)

    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'pickles_files\\saved_weights_'+str(epoch)+'.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%',flush=True)
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n',flush=True)

In [ ]:
#load weights and tokenizer

path='pickles_files\\saved_weights_'+str(epoch)+'.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('pickles_files\\tokenizer_aug.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en_core_web_sm')

def classify_review(review):    
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return pred